# Завдання

Наш продукт – той самий персональний асистент за підпискою </p>
Користувач спочатку потрапляє на сайт (click), де відповідає на питання, 
потім ми формуємо для нього ідеального асистента на основі його відповідей.</p>
В кінці пропонуємо його купити (purchase)</p>
 - В контрольній версії – ціна 9.99 USD per month (6m LTV – 29.27 USD)
 - В тестовій версії – ціна 6.99 USD per month (6m LTV – 23.08 USD)

Проаналізуйте тест, 
скажіть яку ціну
лишаємо? 
На якому етапі ви б
прийняли рішення?

## Розв'язання:

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [3]:
df = pd.read_csv('pricing test data.csv')
df

,user id,event time,event name,group,country,source
0,56164500000000000,02/01/2023,click,test(6.99),Canada,NaN
1,56195200000000000,02/01/2023,click,test(6.99),Chile,NaN
2,56198900000000000,02/01/2023,click,test(6.99),Colombia,NaN
3,56095700000000000,02/01/2023,click,test(6.99),Italy,NaN
4,5616300000000000,02/01/2023,click,test(6.99),Mauritius,NaN
...,...,...,...,...,...,...
96339,56087800000000000,31/1/2023,purchase,test(6.99),United States,1.0
96340,56366200000000000,02/03/2023,purchase,test(6.99),Denmark,1.0
96341,56366200000000000,02/03/2023,purchase,test(6.99),Denmark,1.0
96342,5607900000000000,17/3/2023,purchase,control(9.99),United States,1.0


In [4]:
df['source'] = df['source'].fillna(0)
df['event time'] = pd.to_datetime(df['event time'], format="%d/%m/%Y")


розглянемо розподіл кількості відвідувачів по групах

In [5]:
df.groupby(['group','event name']).count()

user id  event time  country  source
group         event name                                      
control(9.99) click         45878       45878    45874   45878
              purchase       2639        2639     2639    2639
test(6.99)    click         44910       44910    44903   44910
              purchase       2917        2917     2917    2917

In [6]:
df.nunique()

user id       11887
event time      173
event name        2
group             2
country         206
source           16
dtype: int64

In [7]:
df.groupby(['group','event name']).nunique()

user id  event time  country  source
group         event name                                      
control(9.99) click          8826         168      193      14
              purchase       1499          69       96      10
test(6.99)    click          8503         158      192      14
              purchase       1608          59       91      12

Ми працюэмо з двома схожими за кількісним мкладом групами

In [8]:
df.groupby(['event time', 'group'])['user id'].count()

event time  group        
2023-01-02  control(9.99)    6601
            test(6.99)       7023
2023-01-03  control(9.99)      27
            test(6.99)         15
2023-01-04  control(9.99)      12
                             ... 
2023-12-05  control(9.99)      10
            test(6.99)         12
2023-12-06  control(9.99)       3
            test(6.99)          3
2023-12-07  control(9.99)       4
Name: user id, Length: 326, dtype: int64

Додамо конверсію по дням до нашого датасету:</p>
`conversion = count(purchase)/count(click)` 

In [9]:
df

,user id,event time,event name,group,country,source
0,56164500000000000,2023-01-02,click,test(6.99),Canada,0.0
1,56195200000000000,2023-01-02,click,test(6.99),Chile,0.0
2,56198900000000000,2023-01-02,click,test(6.99),Colombia,0.0
3,56095700000000000,2023-01-02,click,test(6.99),Italy,0.0
4,5616300000000000,2023-01-02,click,test(6.99),Mauritius,0.0
...,...,...,...,...,...,...
96339,56087800000000000,2023-01-31,purchase,test(6.99),United States,1.0
96340,56366200000000000,2023-03-02,purchase,test(6.99),Denmark,1.0
96341,56366200000000000,2023-03-02,purchase,test(6.99),Denmark,1.0
96342,5607900000000000,2023-03-17,purchase,control(9.99),United States,1.0


In [10]:
grouped_data =df.groupby(['event time', 'group','event name'])['user id'].count().unstack()
grouped_data

event name                 click  purchase
event time group                          
2023-01-02 control(9.99)  6185.0     416.0
           test(6.99)     6579.0     444.0
2023-01-03 control(9.99)    26.0       1.0
           test(6.99)       12.0       3.0
2023-01-04 control(9.99)    12.0       NaN
...                          ...       ...
2023-12-05 control(9.99)    10.0       NaN
           test(6.99)       11.0       1.0
2023-12-06 control(9.99)     2.0       1.0
           test(6.99)        3.0       NaN
2023-12-07 control(9.99)     4.0       NaN

[326 rows x 2 columns]

In [11]:
grouped_data['conversion'] = (grouped_data['purchase'] / grouped_data['click'])
grouped_data['conversion'] = grouped_data['conversion'].fillna(0)
grouped_data = grouped_data.unstack()
grouped_data

event name         click                 purchase               conversion   
group      control(9.99) test(6.99) control(9.99) test(6.99) control(9.99)   
event time                                                                   
2023-01-02        6185.0     6579.0         416.0      444.0      0.067259  \
2023-01-03          26.0       12.0           1.0        3.0      0.038462   
2023-01-04          12.0       23.0           NaN        2.0      0.000000   
2023-01-05           6.0        9.0           NaN        NaN      0.000000   
2023-01-06           2.0        NaN           NaN        NaN      0.000000   
...                  ...        ...           ...        ...           ...   
2023-12-03          12.0       15.0           1.0        NaN      0.083333   
2023-12-04          11.0       10.0           NaN        1.0      0.000000   
2023-12-05          10.0       11.0           NaN        1.0      0.000000   
2023-12-06           2.0        3.0           1.0        NaN      0.500000   
2023-12-07           4.0        NaN           NaN        NaN      0.000000   

event name             
group      test(6.99)  
event time             
2023-01-02   0.067487  
2023-01-03   0.250000  
2023-01-04   0.086957  
2023-01-05   0.000000  
2023-01-06        NaN  
...               ...  
2023-12-03   0.000000  
2023-12-04   0.100000  
2023-12-05   0.090909  
2023-12-06   0.000000  
2023-12-07        NaN  

[173 rows x 6 columns]

додамо показник різниці конверсії тестового та контрольного набору

In [12]:
grouped_data['conversion_diff'] = grouped_data['conversion']['test(6.99)'] - grouped_data['conversion']['control(9.99)']
grouped_data


event name         click                 purchase               conversion   
group      control(9.99) test(6.99) control(9.99) test(6.99) control(9.99)   
event time                                                                   
2023-01-02        6185.0     6579.0         416.0      444.0      0.067259  \
2023-01-03          26.0       12.0           1.0        3.0      0.038462   
2023-01-04          12.0       23.0           NaN        2.0      0.000000   
2023-01-05           6.0        9.0           NaN        NaN      0.000000   
2023-01-06           2.0        NaN           NaN        NaN      0.000000   
...                  ...        ...           ...        ...           ...   
2023-12-03          12.0       15.0           1.0        NaN      0.083333   
2023-12-04          11.0       10.0           NaN        1.0      0.000000   
2023-12-05          10.0       11.0           NaN        1.0      0.000000   
2023-12-06           2.0        3.0           1.0        NaN      0.500000   
2023-12-07           4.0        NaN           NaN        NaN      0.000000   

event name            conversion_diff  
group      test(6.99)                  
event time                             
2023-01-02   0.067487        0.000228  
2023-01-03   0.250000        0.211538  
2023-01-04   0.086957        0.086957  
2023-01-05   0.000000        0.000000  
2023-01-06        NaN             NaN  
...               ...             ...  
2023-12-03   0.000000       -0.083333  
2023-12-04   0.100000        0.100000  
2023-12-05   0.090909        0.090909  
2023-12-06   0.000000       -0.500000  
2023-12-07        NaN             NaN  

[173 rows x 7 columns]

In [13]:
grouped_data['conversion_diff'].mean()

-0.012474440687949425

розглянемо загальну конверсію за весь час для тестового та контрольного наборів

In [14]:
test_clicks = df[(df['event name'] == 'click') & (df['group'] == 'test(6.99)')]['user id'].count()
test_purchase =  df[(df['event name'] == 'purchase') & (df['group'] == 'test(6.99)')]['user id'].count()
test_conversion = test_purchase/test_clicks
test_conversion


0.06495212647517257

In [15]:
control_clicks = df[(df['event name'] == 'click') & (df['group'] == 'control(9.99)')]['user id'].count()
control_purchase =  df[(df['event name'] == 'purchase') & (df['group'] == 'control(9.99)')]['user id'].count()
control_conversion = control_purchase/control_clicks
control_conversion

0.05752212389380531

In [16]:
test_conversion/control_conversion

1.1291677371837692

отже в загальному конверсія на тестовому наборі збільшилась у 1,13 рази

In [17]:
control_counts = df[df['group'] == 'control(9.99)'].groupby(['event time', 'event name'])['user id'].count().unstack().fillna(0)
control_counts['control_conversion'] = control_counts['purchase']/control_counts['click']


In [18]:
test_counts = df[df['group'] == 'test(6.99)'].groupby(['event time', 'event name'])['user id'].count().unstack().fillna(0)
test_counts['test_conversion'] = test_counts['purchase']/test_counts['click']


In [19]:
test_counts['test_conversion'].mean()

0.0471695408237874

In [20]:
control_counts['control_conversion'].mean()

0.06564311632191273

In [21]:
merged_df = control_counts[['control_conversion']].merge(test_counts[['test_conversion']], left_index=True, right_index=True)

In [22]:
merged_df['conversion_diff'] = merged_df['test_conversion'] - merged_df['control_conversion']
merged_df

event name,control_conversion,test_conversion,conversion_diff
event time,,,
2023-01-02,0.067259,0.067487,0.000228
2023-01-03,0.038462,0.250000,0.211538
2023-01-04,0.000000,0.086957,0.086957
2023-01-05,0.000000,0.000000,0.000000
2023-01-07,0.000000,0.000000,0.000000
...,...,...,...
2023-12-02,0.037500,0.088889,0.051389
2023-12-03,0.083333,0.000000,-0.083333
2023-12-04,0.000000,0.100000,0.100000


In [23]:
print(merged_df['conversion_diff'].mean())
print(merged_df['control_conversion'].mean())
print(merged_df['test_conversion'].mean())

-0.012474440687949425
0.0611854697739521
0.04871102908600268


якщо розглянути середню конверсію по кожному дню, то вона буде меншою за контрольну. Так сталося через нерівномірність розподілення конверсії на тестових та контрольних даних по днях

Отже, в загальному при зменьшені ціни тарифу з 9.99$ до 6.99$ , ми отримаємо у 1,13 разів більше продажів. Це непоганий результат, проте нам потрібно розрахувати чи буде від цього рішення компанія заробляти більше. Ми маємо LTV для кожного з випадків:
- В контрольній версії – ціна 9.99 USD per month (6m LTV – 29.27 USD)
- В тестовій версії – ціна 6.99 USD per month (6m LTV – 23.08 USD) </p>
Отже, що при зменшенні ціни тарифу, ми будемо мати у 1.13 рази більше користувачів, але заробляти з одного клієнта у 23.08/29.27 = 0.789 рази менше </p>
Таким чином загальний дохід зміниться у 1.13*0.789 = 0.892 разів, тобто впаде на 10%

### Висновок:

В результаті зниження ціни на підписку, ми отримаємо більше клієнтів, проте цього не вистачить щоб покрити різниці у ціні на підписку. Враховуючі ці фактори, компанія в результаті зниження ціни втратить 10% свого прибутку. Тому, я вважаю, що ціну на підписку варто залишити на рівні 9.99$